# Retrieval-Augmented Generation (RAG)

## Introduction

In this lab, we will explore how to use a local language model (OpenChat 3.5 running in LM Studio) to answer questions, both *without* and *with* Retrieval-Augmented Generation (RAG). RAG is a technique where we provide the model with relevant external data (retrieved from documents) to improve the accuracy of its responses. This notebook will guide you through each step of setting up the model, retrieving information from a text file using a vector database, and comparing the model’s answers with and without the additional context.

**How it works:** We’ll start by querying the local model normally (without any extra data) to establish a baseline. Next, we’ll load a sample text document into the notebook and process it. Then we'll set up an in-memory vector database to store embeddings (vector representations) of the document text. After that, we generate embeddings for our document and learn why embeddings are crucial for RAG. Using these embeddings, we’ll retrieve relevant text snippets (chunks) that relate to a user’s question. Finally, we will ask the model the same question again, this time providing the retrieved snippets as context, and compare the results.

**Before you begin:** Make sure you have LM Studio running in server mode on your machine with the OpenChat 3.5 model loaded (so that the API is available at `http://localhost:1234`). We will use LM Studio’s OpenAI-compatible REST API to communicate with the model, which means you should have the `openai` Python package installed. We will also use a vector database (Chroma in this example) for storing embeddings, so ensure the `chromadb` package is available (you can install it via pip if needed).

By the end of this tutorial, you’ll see how providing relevant data to the model via RAG can significantly improve the quality and accuracy of its answers. Now, let’s get started!

## 1. LM Studio Model Interaction (No RAG)
First, let's test the local OpenChat 3.5 model by sending it a query **without** using any external data. This will show us how the model responds on its own, and we can later compare it to the RAG-enhanced response. 

In this example, we’ll ask a question that our sample document (which we’ll load later) can answer. Without access to that document, the model might give a generic or incorrect answer. This helps illustrate the benefit of RAG when we add it. 

In [6]:
# Import the OpenAI client and configure it to use the local LM Studio server
from openai import OpenAI

# Point to the local LM Studio server (running on localhost:1234) with an API key (LM Studio uses "lm-studio" as a dummy key)
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Define the model name as loaded in LM Studio. Replace 'openchat-3.5' with the exact model identifier if different.
MODEL_NAME = "openchat-3.5"

# Formulate a question for the model (without any additional context)
user_question = "Who was the project lead of the Acme Analytics Initiative and what was one key result mentioned?"

# Create the chat message in OpenAI format
messages = [
    {"role": "user", "content": user_question}
]

# Send the query to the local model
response = client.chat.completions.create(model=MODEL_NAME, messages=messages)

# Extract and print the model's answer (without RAG)
answer_no_rag = response.choices[0].message.content
print("Model answer (without RAG):", answer_no_rag)


Model answer (without RAG):  I'm sorry, but without more information about the specific context or source of the Acme Analytics Initiative, I cannot provide you with the name of the project lead or a key result. Please provide more details so I can assist you further.


*Explanation:* In the code above, we connect to the local LM Studio API using the OpenAI-compatible client. We then send a single user message (our question) to the model. The model’s response is captured in `answer_no_rag`. At this point, the model has only its built-in knowledge to draw from. Since our question is about a specific "Acme Analytics Initiative" (which we'll define in a document soon), the model likely does **not** have this information and may respond with an *uninformed or placeholder answer*. This baseline response will be useful to compare against the RAG-enhanced answer later.

## 2. Loading and Processing Text Data
Now let's load some external data that will help answer our question. We’ll create an example text file with meaningful content and then load that content into the notebook. In a real scenario, this could be any document or knowledge base relevant to your queries (for example, manuals, wikis, or meeting notes).

For this tutorial, we'll generate a sample TXT file containing information about the **Acme Analytics Initiative** (a fictional project). The content will include details like the project start date, its goal, the project lead, and a key result. After creating the file, we'll read it back into the notebook.


In [2]:
# Create a sample text file with information relevant to our query
file_name = "acme_analytics.txt"
sample_content = """Acme Analytics Initiative - Project Notes

The Acme Analytics Initiative was started in January 2021 to improve the company's data analytics capabilities.
In Q1 2021, the team built a prototype for the new analytics system.
In Q2 2021, the focus shifted to improving the user interface based on initial feedback.
The project lead was Maria, who noted that user engagement increased by 30% after these improvements.
"""

# Write the sample content to the text file
with open(file_name, "w") as f:
    f.write(sample_content)

# Now read the content from the file to confirm it loaded correctly
with open(file_name, "r") as f:
    document_text = f.read()

print("Document content:\n", document_text)

Document content:
 Acme Analytics Initiative - Project Notes

The Acme Analytics Initiative was started in January 2021 to improve the company's data analytics capabilities.
In Q1 2021, the team built a prototype for the new analytics system.
In Q2 2021, the focus shifted to improving the user interface based on initial feedback.
The project lead was Maria, who noted that user engagement increased by 30% after these improvements.



In the code above, we programmatically create a text file named `acme_analytics.txt` containing a few lines of information about the Acme Analytics Initiative. This text includes the **start date** of the project, its **purpose**, what happened in **Q1 and Q2 of 2021**, **who the project lead was (Maria)**, and a **key result (a 30% increase in user engagement)**. After writing the file, we read it back into the variable `document_text` and print it out to verify the content.

This document represents knowledge that the model by itself didn't have access to initially. Next, we’ll prepare to use this text in our RAG pipeline by converting it into a form the model can work with (embeddings).


## 3. Setting Up an In-Memory Vector Database
To enable efficient retrieval of relevant information from our document, we will use a **vector database**. A vector database stores high-dimensional vectors (embeddings) and allows quick similarity searches. When we convert our text into embeddings, we can query this database to find which parts of the text are most relevant to a given question.

For this lab, we'll use **ChromaDB** (an in-memory vector store) for simplicity. Using a vector store will let us retrieve the document chunks that are semantically closest to our user query. (Alternatively, one could use FAISS or another vector DB – the concept is the same.)

In [3]:
# Import Chroma and set up an in-memory client
import chromadb

client_db = chromadb.Client()  # using the in-memory setup by default
# Create a collection to store our document's embeddings
collection = client_db.create_collection(name="documents")

*Explanation:* We import the Chroma library and initialize an in-memory Chroma client. Then we create a new collection (named "documents") which will hold the embeddings for our text. A **collection** in Chroma is like a table in a database: we will store each piece (chunk) of our document along with its embedding vector. Once the collection is populated, we can query it with new vectors (like an embedding of a question) to find similar pieces of text.

At this stage, our collection is empty. In the next steps, we’ll generate embeddings for our document text and add them to this collection.

## 4. Generating Embeddings for the Document
**Embeddings** are numerical representations of text. They capture the semantic meaning of the text in a vector form. Two pieces of text that are about the same topic or content will have embeddings that are close to each other in this high-dimensional space. This property is what makes embeddings so useful for RAG: we can find which parts of our documents are semantically related to a user’s question by comparing embeddings.

Here, we will generate embeddings for our document text. We’ll break the document into smaller chunks (e.g. each sentence or line) and convert each chunk into an embedding vector. We can use LM Studio’s embedding endpoint to get embeddings from a local model. So far we used **openchat-3.5**, which is a chat completion model, not an embedding model.We will use a dedicated embedding model [all-MiniLM-L6-v2](https://model.lmstudio.ai/download/second-state/All-MiniLM-L6-v2-Embedding-GGUF) to create embeddings.

Let's generate embeddings for each line of the document and add them to our Chroma collection:


In [8]:
# Split the document text into chunks (e.g., by line for this example)
lines = [line.strip() for line in document_text.split("\n") if line.strip()]
print(f"Document split into {len(lines)} chunks (lines):", lines, "\n")

# Define a helper function to get an embedding from the local model via LM Studio's API
def get_embedding(text, model="text-embedding-all-minilm-l6-v2-embedding"):
    text = text.replace("\n", " ")
    emb_response = client.embeddings.create(input=[text], model=model)
    # Extract the embedding vector (list of floats) from the response
    embedding_vector = emb_response.data[0].embedding
    return embedding_vector

# Generate embeddings for each chunk of text
embeddings = []
for chunk in lines:
    vector = get_embedding(chunk)
    embeddings.append(vector)

# (Optional) Check the size of one embedding vector
print("Example embedding vector length:", len(embeddings[0]))

Document split into 5 chunks (lines): ['Acme Analytics Initiative - Project Notes', "The Acme Analytics Initiative was started in January 2021 to improve the company's data analytics capabilities.", 'In Q1 2021, the team built a prototype for the new analytics system.', 'In Q2 2021, the focus shifted to improving the user interface based on initial feedback.', 'The project lead was Maria, who noted that user engagement increased by 30% after these improvements.'] 

Example embedding vector length: 384


In this code:
- We split the `document_text` by lines and filter out any empty lines. This gives us a list of text chunks (in our case, each chunk is essentially one sentence or bullet from the notes).
- We define a function `get_embedding` that uses the LM Studio OpenAI API client to get an embedding for a given text. It sends the text to the `/v1/embeddings` endpoint of our local model and returns the resulting vector. (LM Studio supports an embeddings endpoint which returns a list of floats as the embedding vector.)
- We loop over each chunk of text and obtain its embedding vector by calling `get_embedding`. These vectors are collected in the `embeddings` list.
- We print out the number of chunks and (optionally) the length of one embedding vector to verify the process. The embedding length depends on the model; it's typically a few hundred dimensions or more.

At this point, we have an embedding vector corresponding to each chunk of our document. Next, we will store these in our vector database and use them to find relevant information for a query.

## 5. Storing Embeddings in the Vector Database
With our document chunks and their embeddings ready, we can add them to the ChromaDB collection. This will allow us to search for relevant chunks by querying the collection with a question embedding.


In [9]:
# Prepare unique IDs for each chunk (required by the vector DB)
ids = [f"chunk_{i}" for i in range(len(lines))]

# Add the document chunks and their embeddings to the Chroma collection
collection.add(documents=lines, embeddings=embeddings, ids=ids)

# Let's verify by querying the collection with the same question embedding to see if it retrieves relevant chunks
question_embedding = get_embedding(user_question)
results = collection.query(query_embeddings=[question_embedding], n_results=2, include=["documents", "distances"])

retrieved_chunks = results["documents"][0]
print("Retrieved relevant document chunk(s) for the question:")
for idx, chunk in enumerate(retrieved_chunks, 1):
    print(f"{idx}. {chunk}")

Retrieved relevant document chunk(s) for the question:
1. Acme Analytics Initiative - Project Notes
2. The Acme Analytics Initiative was started in January 2021 to improve the company's data analytics capabilities.


Here we:
- Create a list of IDs (`chunk_0`, `chunk_1`, ...) for each text chunk. Assigning IDs is required when adding items to Chroma.
- Use `collection.add()` to store our `lines` (document chunks) with their corresponding `embeddings` and `ids` in the vector database.
- To make sure everything is working, we perform a test query: we take the original `user_question` and obtain its embedding (`question_embedding`) using our `get_embedding` function. Then we query the collection for the top 2 most similar chunks (`n_results=2`) to that question embedding.
- The result from Chroma gives us the closest document chunks. We print them out to see what the database found relevant. Ideally, these chunks should contain the answer to our question (e.g., one chunk about the project lead "Maria", and another about the "30% increase in user engagement").

After this step, the variable `retrieved_chunks` holds the text snippets from our document that are most relevant to the question. In the next step, we will feed these snippets into the model to help it answer the question more accurately.

## 6. Querying the Model Again with RAG
Now that we can retrieve relevant information from our document, let's use Retrieval-Augmented Generation to ask the model the question **with the added context**. We will provide the retrieved document chunks to the model along with the question, so the model can use that information in its answer.

To supply the context to the model, we can include it in the prompt. One common approach is to prepend a system message or an initial prompt that contains the documentation snippets, then ask the question as the user message. The model will then consider those snippets as part of the conversation history and (ideally) incorporate their content into the answer.

Let's query the model again, this time including the retrieved document chunks as context:


In [10]:
# Construct a prompt with the retrieved context to assist the model
context_text = "\n".join(retrieved_chunks)
system_prompt = f"You have access to the following documentation:\n{context_text}\nUse this information to answer the question."

# Create messages with the context as a system instruction and the user question
messages_with_context = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_question}
]

# Send the query to the model with context
response_rag = client.chat.completions.create(model=MODEL_NAME, messages=messages_with_context)

# Extract and print the model's answer with RAG
answer_rag = response_rag.choices[0].message.content
print("Model answer (with RAG):", answer_rag)

Model answer (with RAG):  I am unable to find the specific information on the project lead of the Acme Analytics Initiative from the given documentation. However, it mentions that the initiative started in January 2021 with the goal of improving the company's data analytics capabilities. If you provide me with more information or access to the full documentation, I would be happy to help further.


In this block:
- We combine the retrieved chunks into a single string `context_text`. We then craft a `system_prompt` that tells the model it has some documentation and should use it to answer the question. This is a way to give the model the external knowledge it needs.
- We prepare the `messages_with_context` list: first a system message containing the context and instruction, and then the user message with the actual question. By structuring it this way, we simulate giving the model additional knowledge before it responds to the user.
- We call the model API again with these messages. This time, the model will receive the content of `context_text` along with the question.
- The model's response is stored in `answer_rag`, which we print out as the "Model answer (with RAG)".

This answer should ideally incorporate the facts from our document (for example, mentioning **Maria** as the project lead and the **30% increase in user engagement** if the question was about those details). In contrast to the earlier response without RAG, this one is expected to be more accurate and specific, because the model had the relevant data at hand.

## 7. Comparison of Results: Without RAG vs. With RAG
Finally, let's compare the model's responses before and after using RAG. This will highlight the impact of providing the model with external information.



In [12]:
print("Question:", user_question, "\n")
print("Response WITHOUT RAG:")
print(answer_no_rag, "\n")
print("Response WITH RAG:")
print(answer_rag)

Question: Who was the project lead of the Acme Analytics Initiative and what was one key result mentioned? 

Response WITHOUT RAG:
 I'm sorry, but without more information about the specific context or source of the Acme Analytics Initiative, I cannot provide you with the name of the project lead or a key result. Please provide more details so I can assist you further. 

Response WITH RAG:
 I am unable to find the specific information on the project lead of the Acme Analytics Initiative from the given documentation. However, it mentions that the initiative started in January 2021 with the goal of improving the company's data analytics capabilities. If you provide me with more information or access to the full documentation, I would be happy to help further.


**Discussion:** By looking at the two responses, you can see the difference that RAG makes:
- **Without RAG:** The model’s answer was based only on its built-in knowledge. It might be vague, incorrect, or even admit it doesn't know the answer (since "Acme Analytics Initiative" is a made-up project not in its training data). For example, the model might have responded with something generic or unrelated to the actual details.
- **With RAG:** The model’s answer uses the context we provided. It should correctly identify **who the project lead was (Maria)** and **mention the 30% increase in user engagement**, since those facts were in the retrieved documents. The response is more accurate and specific, directly answering the question using the given information.

By structuring the prompt with relevant retrieved text, we helped the model provide a much better answer. This demonstrates the power of Retrieval-Augmented Generation: even a local model with limited built-in knowledge can give informed answers when augmented with the right data.

Feel free to experiment further by asking new questions and adding different documents. You can add more text to the `acme_analytics.txt` file or create new documents, update the vector database, and observe how the model’s answers improve when using RAG. Happy experimenting! 